# Sales Dashboard ETL

In [106]:
import kagglehub
import pandas as pd

In [107]:
path = kagglehub.dataset_download("kyanyoga/sample-sales-data")

In [108]:
file_path = os.path.join(path, "sales_data_sample.csv")
df = pd.read_csv(file_path, encoding="ISO-8859-1")
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,Shipped,4,12,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Small
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,Shipped,1,1,2005,...,Torikatu 38,NaN,Oulu,NaN,90110,Finland,EMEA,Koskitalo,Pirkko,Medium
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,Resolved,1,3,2005,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Medium
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,Shipped,1,3,2005,...,1 rue Alsace-Lorraine,NaN,Toulouse,NaN,31000,France,EMEA,Roulet,Annette,Small


### Renaming columns

In [110]:
# renaming colunns for better readability
df.columns = [col.lower() for col in df.columns]

new_col_names = [
    'order_number', 'quantity_ordered', 'price_each', 'order_line_number', 'sales',
    'order_date', 'status', 'qtr_id', 'month_id', 'year_id', 'product_line', 'msrp',
    'product_code', 'customer_name', 'phone', 'address_line_1', 'address_line_2',
    'city', 'state', 'postal_code', 'country', 'territory', 'contact_last_name',
    'contact_first_name', 'deal_size'
]

rename_map = {}
old_cols_lower = [col.lower() for col in df.columns]

for old_col, new_col in zip(old_cols_lower, new_col_names):
    # Strip underscores from new col and compare with old col (lowercase)
    if old_col == new_col.replace("_", ""):
        rename_map[df.columns[old_cols_lower.index(old_col)]] = new_col
    

# Apply renaming
df = df.rename(columns=rename_map)
print(df.columns.tolist())


['order_number', 'quantity_ordered', 'price_each', 'order_line_number', 'sales', 'order_date', 'status', 'qtr_id', 'month_id', 'year_id', 'product_line', 'msrp', 'product_code', 'customer_name', 'phone', 'address_line_1', 'address_line_2', 'city', 'state', 'postal_code', 'country', 'territory', 'contact_last_name', 'contact_first_name', 'deal_size']


## Step 1: Investigating the Data

When starting with a new dataset, it is crucial to understand its structure.  

The main checks I perform are:

1. **Check for Duplicate Rows**  
   - Identify if the same `order_number` or `product_code` appears multiple times.  
   - Duplicates can indicate multiple line items per order,changes or updates to the same ID over time  

2. **Determine Field Uniqueness per Group**  
   - Investigate which fields are **constant** within a group (e.g., within the same `order_number` or `customer_name`).  
   - This helps to:  
     - **Normalize the data** into dimension tables  
     - Decide which fields belong in `customer`, `orders`, `product`, or in the **fact table**  


In [114]:

def constant_fields(df, group_field):
    # Identify columns that have the same value within each group
    constant_cols = []
    for col in df.columns:
        if col == group_field:
            continue
        # For each group, check if the column has >1 unique value
        if (df.groupby(group_field)[col].nunique() <= 1).all():
            constant_cols.append(col)
    return constant_cols

# Check constant fields per grouping level
customer_constant = constant_fields(df, 'customer_name')
order_constant = constant_fields(df, 'order_number')
product_constant = constant_fields(df, 'product_code')

print(" Fields constant per customer_name")
print(customer_constant)
print("\n Fields constant per order_number")
print(order_constant)
print("\n Fields constant per product_code")
print(product_constant)


 Fields constant per customer_name
['phone', 'address_line_1', 'address_line_2', 'city', 'state', 'postal_code', 'country', 'territory', 'contact_last_name', 'contact_first_name']

 Fields constant per order_number
['order_date', 'status', 'qtr_id', 'month_id', 'year_id', 'customer_name', 'phone', 'address_line_1', 'address_line_2', 'city', 'state', 'postal_code', 'country', 'territory', 'contact_last_name', 'contact_first_name']

 Fields constant per product_code
['product_line', 'msrp']


In [116]:
# Investigate what 'deal_size' represents 
deal_size_check = (
    df.groupby('order_number')['deal_size'].nunique()
)
inconsistent_orders = deal_size_check[deal_size_check > 1]
inconsistent_orders_sorted = inconsistent_orders.sort_values(ascending=False)

print(f"Orders with multiple deal sizes: {len(inconsistent_orders_sorted)}")
print(inconsistent_orders_sorted.head(10))


Orders with multiple deal sizes: 277
order_number
10263    3
10226    3
10214    3
10356    3
10217    3
10219    3
10220    3
10223    3
10227    3
10182    3
Name: deal_size, dtype: int64


**Conclusion:**  
We can see that `deal_size` is **not constant within an order**—it varies across the products in the same order.  
For example, 277 orders have multiple `deal_size` values, and several orders (e.g., 10263, 10226) have **3 different deal sizes** within a single order.

## Step 2: Creating a Unique ID for Each Client

It is recommended **not to rely on `customer_name`** as the key for a schema because:

1. **Numeric IDs** are preferred over text for better performance and consistency.  
2. **Customer names are not guaranteed to be unique**- two different clients can share the same name.  

To address this, a **unique `customer_id`** is generated based on all the customer details.  
This ensures that the same client always receives the same ID, and different clients with the same name receive different IDs.


In [117]:
from pandas.util import hash_pandas_object

# 1. Define customer fields
customer_fields = [
    'customer_name', 'phone', 'address_line_1', 'address_line_2',
    'city', 'state', 'postal_code', 'country', 'territory',
    'contact_last_name', 'contact_first_name'
]

# 2. Generate stable numeric customer_id
#    hash_pandas_object ensures stable IDs for identical rows across runs
df['customer_id'] = hash_pandas_object(df[customer_fields], index=False).astype('uint64')
df[['customer_id','customer_name', 'phone', 'address_line_1', 'address_line_2',
    'city', 'state', 'postal_code', 'country', 'territory',
    'contact_last_name', 'contact_first_name']]

,customer_id,customer_name,phone,address_line_1,address_line_2,city,state,postal_code,country,territory,contact_last_name,contact_first_name
0,6452243583427845017,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai
1,2972231027799034877,Reims Collectables,26.47.1555,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul
2,13641214029300543396,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel
3,17608339215448200492,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie
4,3615626713207920220,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie
...,...,...,...,...,...,...,...,...,...,...,...,...
2818,3261117559738986018,Euro Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego
2819,11182691218010391523,"Oulu Toy Supplies, Inc.",981-443655,Torikatu 38,NaN,Oulu,NaN,90110,Finland,EMEA,Koskitalo,Pirkko
2820,3261117559738986018,Euro Shopping Channel,(91) 555 94 44,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego
2821,10957316647086540047,Alpha Cognac,61.77.6555,1 rue Alsace-Lorraine,NaN,Toulouse,NaN,31000,France,EMEA,Roulet,Annette


## Step 3: Creating the Tables Structure

In this step, I transform the cleaned data into a schema for the Tableau dashboard.    
I use the fields that are **constant per group** (as identified in the data investigation step, per `customer_id`, `order_number`, or `product_code`) to **minimize duplicates**.  
By moving these constant fields into dimension tables, I reduce repeated values and make the dataset **smaller, cleaner, and more efficient** for BI tools.


In [119]:
# based on the fields I examined earlier, I want to minimize redundancy and duplication as possible.

# 1. CUSTOMER TABLE
customer_fields = [
    'customer_id', 'customer_name', 'phone', 'address_line_1', 'address_line_2',
    'city', 'state', 'postal_code', 'country', 'territory',
    'contact_last_name', 'contact_first_name'
]

Customer = df[customer_fields].drop_duplicates().reset_index(drop=True)


# 2. ORDERS TABLE (uses customer_id as FK)
orders_fields = [
    'order_number', 'order_date', 'status', 'customer_id'
]

Order = df[orders_fields].drop_duplicates().reset_index(drop=True)


# 3. PRODUCT TABLE
product_fields = ['product_code', 'product_line', 'msrp']
Product = df[product_fields].drop_duplicates().reset_index(drop=True)


# 4. ORDERS_AND_PRODUCTS FACT TABLE
orders_and_product_fields = [
    'order_number', 'product_code', 'order_line_number', 'quantity_ordered', 
    'price_each', 'sales', 'deal_size'
]
Orders_Products = df[orders_and_product_fields].drop_duplicates().reset_index(drop=True)
#generating an order_product key (useful for trends and measures)
Orders_Products['order_product_key'] = (
    orders_and_products_table['order_number'].astype(str) + '_' + 
    orders_and_products_table['product_code']
)



# 5. Date TABLE
date_fields = ['order_number', 'qtr_id', 'month_id', 'year_id']
Dates = df[date_fields].drop_duplicates().reset_index(drop=True)



In [120]:
Order

,order_number,order_date,status,customer_id
0,10107,2/24/2003 0:00,Shipped,6452243583427845017
1,10121,5/7/2003 0:00,Shipped,2972231027799034877
2,10134,7/1/2003 0:00,Shipped,13641214029300543396
3,10145,8/25/2003 0:00,Shipped,17608339215448200492
4,10159,10/10/2003 0:00,Shipped,3615626713207920220
...,...,...,...,...
302,10199,12/1/2003 0:00,Shipped,13578294288264486670
303,10397,3/28/2005 0:00,Shipped,10957316647086540047
304,10352,12/3/2004 0:00,Shipped,16741429585515729936
305,10118,4/21/2003 0:00,Shipped,10804791857753206855


In [121]:
Product

,product_code,product_line,msrp
0,S10_1678,Motorcycles,95
1,S10_1949,Classic Cars,214
2,S10_2016,Motorcycles,118
3,S10_4698,Motorcycles,193
4,S10_4757,Classic Cars,136
...,...,...,...
104,S700_3505,Ships,100
105,S700_3962,Ships,99
106,S700_4002,Planes,74
107,S72_1253,Planes,49


In [122]:
Customer

,customer_id,customer_name,phone,address_line_1,address_line_2,city,state,postal_code,country,territory,contact_last_name,contact_first_name
0,6452243583427845017,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai
1,2972231027799034877,Reims Collectables,26.47.1555,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul
2,13641214029300543396,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel
3,17608339215448200492,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie
4,3615626713207920220,Corporate Gift Ideas Co.,6505551386,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie
...,...,...,...,...,...,...,...,...,...,...,...,...
87,14328738306532600828,"Australian Collectables, Ltd",61-9-3844-6555,7 Allen Street,NaN,Glen Waverly,Victoria,3150,Australia,APAC,Connery,Sean
88,18230223445440903668,Gift Ideas Corp.,2035554407,2440 Pompton St.,NaN,Glendale,CT,97561,USA,NaN,Lewis,Dan
89,7804525100089437900,"Bavarian Collectables Imports, Co.",+49 89 61 08 9555,Hansastr. 15,NaN,Munich,NaN,80686,Germany,EMEA,Donnermeyer,Michael
90,12950831857277935632,Royale Belge,(071) 23 67 2555,"Boulevard Tirou, 255",NaN,Charleroi,NaN,B-6000,Belgium,EMEA,Cartrain,Pascale


In [123]:
Orders_Products

,order_number,product_code,order_line_number,quantity_ordered,price_each,sales,deal_size,order_product_key
0,10107,S10_1678,2,30,95.70,2871.00,Small,10107_S10_1678
1,10121,S10_1678,5,34,81.35,2765.90,Small,10121_S10_1678
2,10134,S10_1678,2,41,94.74,3884.34,Medium,10134_S10_1678
3,10145,S10_1678,6,45,83.26,3746.70,Medium,10145_S10_1678
4,10159,S10_1678,14,49,100.00,5205.27,Medium,10159_S10_1678
...,...,...,...,...,...,...,...,...
2818,10350,S72_3212,15,20,100.00,2244.40,Small,10350_S72_3212
2819,10373,S72_3212,1,29,100.00,3978.51,Medium,10373_S72_3212
2820,10386,S72_3212,4,43,100.00,5417.57,Medium,10386_S72_3212
2821,10397,S72_3212,1,34,62.24,2116.16,Small,10397_S72_3212


In [124]:
Dates

,order_number,qtr_id,month_id,year_id
0,10107,1,2,2003
1,10121,2,5,2003
2,10134,3,7,2003
3,10145,3,8,2003
4,10159,4,10,2003
...,...,...,...,...
302,10199,4,12,2003
303,10397,1,3,2005
304,10352,4,12,2004
305,10118,2,4,2003


In [125]:
## Step 4: Exporting to CSV

In [126]:

# Export all 5 tables to CSV
customer_table.to_csv("Customer.csv", index=False)
orders_table.to_csv("Order.csv", index=False)
product_table.to_csv("Product.csv", index=False)
orders_and_products_table.to_csv("Orders_Products.csv", index=False)
composite_key_table.to_csv("Dates.csv", index=False)
